In [22]:
from visual_behavior import database as db
import visual_behavior.data_access.loading as loading
import seaborn as sns

from visual_behavior.utilities import get_response_rates

from allensdk.internal.api import PostgresQueryMixin
from allensdk.core.authentication import credential_injector
from allensdk.core.auth_config import LIMS_DB_CREDENTIAL_MAP

from multiprocessing import Pool

from scipy.interpolate import interp1d
from scipy import stats

%standard_imports
%widescreen
%matplotlib notebook

import warnings
warnings.filterwarnings('ignore')

imported:
	os
	pandas as pd
	numpy as np
	matplotlib.pyplot as plt


In [3]:
def load_flashwise_summary(behavior_session_id=None):
    conn = db.Database('visual_behavior_data')
    collection = conn['behavior_analysis']['annotated_stimulus_presentations']

    if behavior_session_id is None:
        # load all
        df = pd.DataFrame(list(collection.find({})))
    else:
        # load data from one behavior session
        df = pd.DataFrame(list(collection.find({'behavior_session_id':behavior_session_id})))

    conn.close()
    
    return df.sort_values(by=['behavior_session_id','flash_index'])

In [4]:
%%time
flash_summary = load_flashwise_summary()

CPU times: user 16.2 s, sys: 1.85 s, total: 18 s
Wall time: 22.6 s


In [5]:
len(flash_summary)

672177

In [6]:
flash_summary.sample(10)

,_id,flash_index,image_index,image_name,omitted,change,licked,lick_rate,rewarded,reward_rate,running_rate,bout_rate,hit_bout,hit_fraction,hit_rate,miss_rate,false_alarm_rate,correct_reject_rate,d_prime,criterion,high_lick,high_reward,flash_metrics_epochs,flash_metrics_labels,in_bout,bias,omissions1,task0,timing1D,start_time,start_frame,stop_time,epoch,behavior_session_id,criterion_negative,motivation_state,session_cluster_id,session_cluster_name
627128,5e953cd7455f9831fc7f6ffd,4394,6,im085,0,1,0,0.121120,0,0.005755,28.538079,0.093125,0.0,0.061801,0.073963,0.926037,0.069588,0.930412,0.031963,-1.462881,0,0,0,"low-lick,low-reward",0,-3.898532,0.806755,2.156982,-2.300012,3598.270858,215716,3598.521064,50.0,908908629,-1.835757,under_motivated,5.0,Grey Screen Impulsive
60587,5e94f6ae04024a017a7f3c67,1524,7,im069,0,0,0,0.388932,0,0.017552,2.584689,0.300156,0.0,0.058476,1.000000,0.000000,0.214781,0.785219,3.116291,0.768202,1,0,2,"high-lick,low-reward",0,-1.225310,2.063734,1.826573,3.460309,1443.968833,86567,1444.219039,10.0,922467930,0.101456,motivated,0.0,Steady
504048,5e953ca7145278bfdd7f6ffc,323,1,im077,0,0,0,0.438464,0,0.064766,58.861699,0.172682,0.0,0.375057,0.908327,0.091673,0.085510,0.914490,2.699456,-0.019204,1,1,1,"high-lick,high-reward",0,-0.919916,1.484481,3.171421,5.995938,542.499405,32518,542.749617,0.0,884888379,1.973102,over_motivated,2.0,Run Away
6243,5e94f3abef4ce608f61ad5c0,1441,2,im066,0,0,0,0.361120,0,0.044219,0.736610,0.174036,0.0,0.254078,0.630056,0.369944,0.102773,0.897227,1.597909,-0.466954,1,1,1,"high-lick,high-reward",0,-2.022725,1.685222,2.025763,3.501542,1381.676192,82833,1381.926394,10.0,937663417,0.201125,motivated,3.0,Late Lick Spike
187062,5e950831ea9c059da17f3735,4769,1,im106,0,0,0,0.347292,0,0.057370,0.178400,0.172057,0.0,0.295444,0.886064,0.113936,0.095006,0.904994,2.516406,-0.052342,1,1,1,"high-lick,high-reward",0,-3.442829,1.915297,3.721617,3.508629,3879.729860,232592,3879.980048,50.0,931955527,0.320959,motivated,8.0,Early Satiated
626984,5e953cd7afc5c21a387f6de2,4431,5,im062,0,1,1,0.269453,1,0.019531,46.249361,0.141562,1.0,0.123068,0.255088,0.744912,0.097074,0.902926,0.639844,-0.978485,1,0,2,"high-lick,low-reward",0,-2.468890,2.264056,2.245063,0.718396,3626.162268,217355,3626.412459,50.0,848599542,-1.361037,under_motivated,2.0,Run Away
453485,5e95312a3d35f740977f5808,3532,6,im085,0,0,0,0.344818,0,0.016120,0.897744,0.189792,0.0,0.084934,1.000000,0.000000,0.127865,0.872135,3.462890,0.594903,1,0,2,"high-lick,low-reward",0,-1.802661,1.236399,2.498355,2.696912,2951.251942,176924,2951.502094,40.0,960697332,-0.569758,motivated,0.0,Steady
160270,5e94ff0e51c7d0468a7f3b7d,1290,6,im085,0,0,1,0.313984,0,0.016641,1.386407,0.195937,0.0,0.084928,0.717172,0.282828,0.136854,0.863146,1.669022,-0.260051,1,0,2,"high-lick,low-reward",0,-2.835056,1.690230,1.501023,5.058812,1268.311449,76036,1268.561653,10.0,963663505,-0.892476,motivated,NaN,NaN
663226,5e953e060260f1c4f67f44dc,2999,7,im035,0,0,0,0.610677,0,0.060260,48.067687,0.201771,1.0,0.297754,0.996975,0.003025,0.111300,0.888700,3.545993,0.553351,1,1,1,"high-lick,high-reward",0,-1.710447,1.649777,6.135954,-0.556576,2551.151470,152941,2551.401680,30.0,939533442,0.511649,motivated,6.0,Burnout
126774,5e94f9d121f99b1fb17f455c,3817,5,im062,0,0,1,0.051172,0,0.009870,30.825458,0.030312,1.0,0.325601,0.114675,0.885325,0.016390,0.983610,0.932731,-1.668402,0,0,0,"low-lick,low-reward",1,-2.301023,0.710766,1.580335,1.586679,3165.303368,189760,3165.553571,40.0,804971463,-1.660403,under_motivated,NaN,NaN


In [7]:
behavior_session_ids = flash_summary['behavior_session_id'].unique()

In [11]:
filtered_session_table = dl.get_filtered_ophys_sessions_table().drop_duplicates('ophys_session_id')

# pull in pass/fail data from the sdk validation log
data_availability = []
for behavior_session_id in filtered_session_table['behavior_session_id'].unique():
    data_availability.append(sdk_validation.get_validation_results(int(behavior_session_id))[['eye_tracking','running_data_df','stimulus_presentations','rewards']])
data_availability = pd.concat(data_availability).reset_index()

# merge in data availability
filtered_session_table = filtered_session_table.merge(
    data_availability,
    left_on = 'behavior_session_id',
    right_on = 'behavior_session_id',
    how='left'
)

NameError: name 'sdk_validation' is not defined